# Project 1

**Dataset:** Speed dating


In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

pd.set_option("display.max_columns", None)


In [ ]:
import wandb

super_secret_API_key = "d1678741bcd8eb1d90ca26aa8f2bf079256ac391"

wandb.login(key=super_secret_API_key)
# ak to chceme pouzivat, tak si tu budes musiet spravit ucet aby som ti mohol dat pristup
# https://wandb.ai/bizon-mimi-stu/ZNEUS_project1/workspace?nw=nwuserbizonmimi

In [ ]:
# wandb test
import random

# Start a new wandb run to track this script.
run = wandb.init(
    entity="bizon-mimi-stu",  # team name
    project="ZNEUS_project1",  # project name
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

In [119]:
df_dating = pd.read_csv("DATA/speeddating.csv")
df_dating.head()

C:\Users\bizon\AppData\Local\Temp\ipykernel_29056\3295607780.py:1: DtypeWarning: Columns (3,10,11,15,16,17,18,19,39,40,41,42,43,44,51,52,53,54,55,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,107,109) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dating = pd.read_csv("DATA/speeddating.csv")


,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,field,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,d_pref_o_attractive,d_pref_o_sincere,d_pref_o_intelligence,d_pref_o_funny,d_pref_o_ambitious,d_pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,d_attractive_o,d_sinsere_o,d_intelligence_o,d_funny_o,d_ambitous_o,d_shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,d_attractive_important,d_sincere_important,d_intellicence_important,d_funny_important,d_ambtition_important,d_shared_interests_important,attractive,sincere,intelligence,funny,ambition,d_attractive,d_sincere,d_intelligence,d_funny,d_ambition,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,d_attractive_partner,d_sincere_partner,d_intelligence_partner,d_funny_partner,d_ambition_partner,d_shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,d_sports,d_tvsports,d_exercise,d_dining,d_museums,d_art,d_hiking,d_gaming,d_clubbing,d_reading,d_tv,d_theater,d_movies,d_concerts,d_music,d_shopping,d_yoga,interests_correlate,d_interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,0,1,female,21,27,6,[4-6],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,2,4,[2-5],[2-5],Law,35,20,20,20,0,5,[21-100],[16-20],[16-20],[16-20],[0-15],[0-15],6,8,8,8,8,6,[6-8],[6-8],[6-8],[6-8],[6-8],[6-8],15,20,20,15,15,15,[0-15],[16-20],[16-20],[0-15],[0-15],[0-15],6,8,8,8,7,[6-8],[6-8],[6-8],[6-8],[6-8],6,9,7,7,6,5,[6-8],[9-10],[6-8],[6-8],[6-8],[0-5],9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.14,[0-0.33],3,2,4,[0-4],[0-3],[3-5],7,6,[6-8],[5-6],0,1,0,0
1,0,1,female,21,22,1,[0-1],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,2,4,[2-5],[2-5],Law,60,0,0,40,0,0,[21-100],[0-15],[0-15],[21-100],[0-15],[0-15],7,8,10,7,7,5,[6-8],[6-8],[9-10],[6-8],[6-8],[0-5],15,20,20,15,15,15,[0-15],[16-20],[16-20],[0-15],[0-15],[0-15],6,8,8,8,7,[6-8],[6-8],[6-8],[6-8],[6-8],7,8,7,8,5,6,[6-8],[6-8],[6-8],[6-8],[0-5],[6-8],9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.54,[0.33-1],3,2,4,[0-4],[0-3],[3-5],7,5,[6-8],[5-6],1,1,0,0
2,1,1,female,21,22,1,[0-1],'Asian/Pacific Islander/Asian-American','Asian/Pacific Islander/Asian-American',1,2,4,[2-5],[2-5],Law,19,18,19,18,14,12,[16-20],[16-20],[16-20],[16-20],[0-15],[0-15],10,10,10,10,10,10,[9-10],[9-10],[9-10],[9-10],[9-10],[9-10],15,20,20,15,15,15,[0-15],[16-20],[16-20],[0-15],[0-15],[0-15],6,8,8,8,7,[6-8],[6-8],[6-8],[6-8],[6-8],5,8,9,8,5,7,[0-5],[6-8],[9-10],[6-8],[0-5],[6-8],9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.16,[0-0.33],3,2,4,[0-4],[0-3],[3-5],7,?,[6-8],[0-4],1,1,1,1
3,0,1,female,21,23,2,[2-3],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,2,4,[2-5],[2-5],Law,30,5,15,40,5,5,[21-100],[0-15],[0-15],[21-100],[0-15],[0-15],7,8,9,8,9,8,[6-8],[6-8],[9-10],[6-8],[9-10],[6-8],15,20,20,15,15,15,[0-15],[16-20],[16-20],[0-15],[0-15],[0-15],6,8,8,8,7,[6-8],[6-8],[6-8],[6-8],[6-8],7,6,8,7,6,8,[6-8],[6-8],[6-8],[6-8],[6-8],[6-8],9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10]

In [120]:
df_dating.replace("?", np.nan, inplace=True)

In [121]:
df_dating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 123 entries, has_null to match
dtypes: int64(7), object(116)
memory usage: 7.9+ MB


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Get all columns that contain range strings
range_cols = [col for col in df_dating.columns if col.startswith("d_")]
other_categorical = ["gender", "race", "race_o"]

cols_to_encode = range_cols + other_categorical
le = LabelEncoder()

for col in cols_to_encode:
    if col in df_dating.columns:
        if df_dating[col].dtype in ["object", "string"]:
            most_frequent = df_dating[col].mode()[0]
            df_dating[col] = df_dating[col].fillna(most_frequent)

            df_dating[col] = le.fit_transform(df_dating[col])

print("Encoded columns sample:")
print(df_dating[cols_to_encode].head())
print("\nData types:")
print(df_dating[cols_to_encode].dtypes)
print("\nUnique values in d_d_age:", df_dating["d_d_age"].unique())

Encoded columns sample:
   d_age  d_d_age  d_importance_same_race  d_importance_same_religion  \
0      6        2                       1                           1   
1      1        0                       1                           1   
2      1        0                       1                           1   
3      2        1                       1                           1   
4      3        1                       1                           1   

   d_pref_o_attractive  d_pref_o_sincere  d_pref_o_intelligence  \
0                    2                 1                      1   
1                    2                 0                      0   
2                    1                 1                      1   
3                    2                 0                      0   
4                    2                 0                      1   

   d_pref_o_funny  d_pref_o_ambitious  d_pref_o_shared_interests  \
0               1                   0                          0  

In [123]:
df_dating.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,field,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,d_pref_o_attractive,d_pref_o_sincere,d_pref_o_intelligence,d_pref_o_funny,d_pref_o_ambitious,d_pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,d_attractive_o,d_sinsere_o,d_intelligence_o,d_funny_o,d_ambitous_o,d_shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,d_attractive_important,d_sincere_important,d_intellicence_important,d_funny_important,d_ambtition_important,d_shared_interests_important,attractive,sincere,intelligence,funny,ambition,d_attractive,d_sincere,d_intelligence,d_funny,d_ambition,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,d_attractive_partner,d_sincere_partner,d_intelligence_partner,d_funny_partner,d_ambition_partner,d_shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,d_sports,d_tvsports,d_exercise,d_dining,d_museums,d_art,d_hiking,d_gaming,d_clubbing,d_reading,d_tv,d_theater,d_movies,d_concerts,d_music,d_shopping,d_yoga,interests_correlate,d_interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,0,1,0,21,27,6,2,0,3,0,2,4,1,1,Law,35,20,20,20,0,5,2,1,1,1,0,0,6,8,8,8,8,6,1,1,1,1,1,1,15,20,20,15,15,15,0,1,1,0,0,0,6,8,8,8,7,1,1,1,1,1,6,9,7,7,6,5,1,2,1,1,1,0,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2,0,1,2,0,0,0,0,0,1,2,0,2,2,2,1,0,0.14,1,3,2,4,0,0,1,7,6,1,1,0,1,0,0
1,0,1,0,21,22,1,0,0,3,0,2,4,1,1,Law,60,0,0,40,0,0,2,0,0,2,0,0,7,8,10,7,7,5,1,1,2,1,1,0,15,20,20,15,15,15,0,1,1,0,0,0,6,8,8,8,7,1,1,1,1,1,7,8,7,8,5,6,1,1,1,1,0,1,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2,0,1,2,0,0,0,0,0,1,2,0,2,2,2,1,0,0.54,2,3,2,4,0,0,1,7,5,1,1,1,1,0,0
2,1,1,0,21,22,1,0,0,0,1,2,4,1,1,Law,19,18,19,18,14,12,1,1,1,1,0,0,10,10,10,10,10,10,2,2,2,2,2,2,15,20,20,15,15,15,0,1,1,0,0,0,6,8,8,8,7,1,1,1,1,1,5,8,9,8,5,7,0,1,2,1,0,1,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2,0,1,2,0,0,0,0,0,1,2,0,2,2,2,1,0,0.16,1,3,2,4,0,0,1,7,NaN,1,0,1,1,1,1
3,0,1,0,21,23,2,1,0,3,0,2,4,1,1,Law,30,5,15,40,5,5,2,0,0,2,0,0,7,8,9,8,9,8,1,1,2,1,2,1,15,20,20,15,15,15,0,1,1,0,0,0,6,8,8,8,7,1,1,1,1,1,7,6,8,7,6,8,1,1,1,1,1,1,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2,0,1,2,0,0,0,0,0,1,2,0,2,2,2,1,0,0.61,2,3,2,4,0,0,1,7,6,1,1,0,1,1,1
4,0,1,0,21,24,3,1,0,2,0,2,4,1,1,Law,30,10,20,10,10,20,2,0,1,0,0,1,8,7,9,6,9,7,1,1,2,1,2,1,15,20,20,15,15,15,0,1,1,0,0,0,6,8,8,8,7,1,1,1,1,1,5,6,7,7,6,6,0,1,1,1,1,1,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2,0,1,2,0,0,0,0,0,1,2,0,2,2,2,1,0,0.21,1,3,2,4,0,0,1,6,6,1,1,0,1,1,1


In [124]:
df_dating["guess_prob_liked"].unique()

array(['6', '5', nan, '7', '4', '3', '8', '1', '10', '2', '9', '4.5',
       '6.5', '5.5', '0', '7.5', '8.5', '9.5', '3.5', '1.5'], dtype=object)

In [125]:
df_dating.shape

(8378, 123)

In [126]:
df_dating.info

<bound method DataFrame.info of       has_null  wave  gender age age_o  d_age  d_d_age  race  race_o  \
0            0     1       0  21    27      6        2     0       3   
1            0     1       0  21    22      1        0     0       3   
2            1     1       0  21    22      1        0     0       0   
3            0     1       0  21    23      2        1     0       3   
4            0     1       0  21    24      3        1     0       2   
...        ...   ...     ...  ..   ...    ...      ...   ...     ...   
8373         1    21       1  25    26      1        0     3       2   
8374         1    21       1  25    24      1        0     3       4   
8375         1    21       1  25    29      4        2     3       2   
8376         1    21       1  25    22      3        1     3       0   
8377         1    21       1  25    22      3        1     3       0   

      samerace importance_same_race importance_same_religion  \
0            0                    2    

In [127]:
missing_values = df_dating.isna().sum().sort_values(ascending=False)

# Percentage of missing values
missing_percentage = (df_dating.isna().mean() * 100).sort_values(ascending=False)

missing_df = pd.DataFrame(
    {"Missing Values": missing_values, "Percentage": missing_percentage}
)

print(missing_df.head(50))

                               Missing Values  Percentage
expected_num_interested_in_me            6578   78.515159
expected_num_matches                     1173   14.000955
shared_interests_o                       1076   12.843161
shared_interests_partner                 1067   12.735736
ambitous_o                                722    8.617809
ambition_partner                          712    8.498448
met                                       375    4.476009
funny_o                                   360    4.296968
funny_partner                             350    4.177608
guess_prob_liked                          309    3.688231
intelligence_o                            306    3.652423
intelligence_partner                      296    3.533063
sinsere_o                                 287    3.425639
sincere_partner                           277    3.306278
like                                      240    2.864646
attractive_o                              212    2.530437
attractive_par

In [128]:
df_dating = df_dating.drop(columns="expected_num_interested_in_me")
df_dating.shape

(8378, 122)

In [129]:
print("Total NaN values:", df_dating.isna().sum().sum())

print("\nNaN count per column:")
print(df_dating.isna().sum())

print("\nColumns with NaN:")
print(df_dating.isna().sum()[df_dating.isna().sum() > 0])

print("\nPercentage of NaN per column:")
print((df_dating.isna().sum() / len(df_dating)) * 100)

Total NaN values: 11658

NaN count per column:
has_null                0
wave                    0
gender                  0
age                    95
age_o                 104
                     ... 
d_guess_prob_liked      0
met                   375
decision                0
decision_o              0
match                   0
Length: 122, dtype: int64

Columns with NaN:
age                                95
age_o                             104
importance_same_race               79
importance_same_religion           79
field                              63
pref_o_attractive                  89
pref_o_sincere                     89
pref_o_intelligence                89
pref_o_funny                       98
pref_o_ambitious                  107
pref_o_shared_interests           129
attractive_o                      212
sinsere_o                         287
intelligence_o                    306
funny_o                           360
ambitous_o                        722
shared_interes

In [130]:
from sklearn.impute import SimpleImputer

# For numerical columns - use median or mean
imputer_num = SimpleImputer(strategy="median")  # or 'mean'
numeric_cols = df_dating.select_dtypes(include=["float64", "int64"]).columns
df_dating[numeric_cols] = imputer_num.fit_transform(df_dating[numeric_cols])

# For categorical columns - use most frequent
imputer_cat = SimpleImputer(strategy="most_frequent")
categorical_cols = df_dating.select_dtypes(include=["object", "string"]).columns
df_dating[categorical_cols] = imputer_cat.fit_transform(df_dating[categorical_cols])

In [131]:
print("Total NaN values:", df_dating.isna().sum().sum())

print("\nNaN count per column:")
print(df_dating.isna().sum())

print("\nColumns with NaN:")
print(df_dating.isna().sum()[df_dating.isna().sum() > 0])

print("\nPercentage of NaN per column:")
print((df_dating.isna().sum() / len(df_dating)) * 100)

Total NaN values: 0

NaN count per column:
has_null              0
wave                  0
gender                0
age                   0
age_o                 0
                     ..
d_guess_prob_liked    0
met                   0
decision              0
decision_o            0
match                 0
Length: 122, dtype: int64

Columns with NaN:
Series([], dtype: int64)

Percentage of NaN per column:
has_null              0.0
wave                  0.0
gender                0.0
age                   0.0
age_o                 0.0
                     ... 
d_guess_prob_liked    0.0
met                   0.0
decision              0.0
decision_o            0.0
match                 0.0
Length: 122, dtype: float64
